In [2]:
from bs4 import BeautifulSoup
from config import chromedriver_path
from pprint import pprint
from splinter import Browser
import pandas as pd
import requests

In [3]:
executable_path = {'executable_path': chromedriver_path}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
result = {}
url = 'https://mars.nasa.gov/news'
browser.visit(url)
url = browser.url

# Since the page has a redirect to trigger the react.js script, we need to revisit
# the page with the parameters it redirects to
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [5]:
# Now we can use the soup to find the elements we need from the browser's inspector
item_list = soup.find('ul', class_='item_list')
slide = item_list.find('li', class_='slide')
list_text = slide.find(class_='list_text')
content_title = list_text.find(class_='content_title')
article_teaser_body = list_text.find(class_='article_teaser_body')

result['news_title'] = content_title.text
result['news_p'] = article_teaser_body.text

pprint(result)

{'news_p': 'An electronics issue is suspected to be preventing the sensors '
           'from sharing their data about Mars weather with the spacecraft.',
 'news_title': "NASA Engineers Checking InSight's Weather Sensors"}


In [12]:
base_url = 'https://www.jpl.nasa.gov'
url = base_url + '/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

main_feature = soup.find('section', class_='main_feature')
a = main_feature.find('a', id='full_image')

url = base_url + a['data-link']
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

img = soup.find('img', class_='main_image')
src = img['src']

result['featured_image_url'] = base_url + src

pprint(result)

{'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA08097_hires.jpg',
 'news_p': 'An electronics issue is suspected to be preventing the sensors '
           'from sharing their data about Mars weather with the spacecraft.',
 'news_title': "NASA Engineers Checking InSight's Weather Sensors"}


In [22]:
url = 'https://space-facts.com/mars/'
table_dfs = pd.read_html(url)
table_df = table_dfs[0]

table_df.columns = ['Term', 'Description']
table_df.head(10)

result['facts-table'] = table_df.to_html()

pprint(result)


{'facts-table': '<table border="1" class="dataframe">\n'
                '  <thead>\n'
                '    <tr style="text-align: right;">\n'
                '      <th></th>\n'
                '      <th>Term</th>\n'
                '      <th>Description</th>\n'
                '    </tr>\n'
                '  </thead>\n'
                '  <tbody>\n'
                '    <tr>\n'
                '      <th>0</th>\n'
                '      <td>Equatorial Diameter:</td>\n'
                '      <td>6,792 km</td>\n'
                '    </tr>\n'
                '    <tr>\n'
                '      <th>1</th>\n'
                '      <td>Polar Diameter:</td>\n'
                '      <td>6,752 km</td>\n'
                '    </tr>\n'
                '    <tr>\n'
                '      <th>2</th>\n'
                '      <td>Mass:</td>\n'
                '      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n'
                '    </tr>\n'
                '    <tr>\n'
                '      <t

In [30]:
base_url = 'https://astrogeology.usgs.gov'
url = base_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

product_section = soup.find(id='product-section')
items = product_section.find_all(class_='item')

hemisphere_image_urls = []
for item in items:
    a = item.find('a', class_='itemLink product-item')
    link = a['href']
        
    browser.visit(base_url + link)
    soup = BeautifulSoup(browser.html, 'html.parser')
    
    wide_image = soup.find(id='wide-image')
    img = wide_image.find('img', class_='wide-image')
    img_src = img['src']
    
    title = soup.find('h2', class_='title').text
    title = title.replace(' Enhanced', '')
    
    hemisphere_image_urls.append({
        'title': title,
        'img-url': base_url + img_src
    })

result['hemisphere_image_urls'] = hemisphere_image_urls

pprint(result)

{'facts-table': '<table border="1" class="dataframe">\n'
                '  <thead>\n'
                '    <tr style="text-align: right;">\n'
                '      <th></th>\n'
                '      <th>Term</th>\n'
                '      <th>Description</th>\n'
                '    </tr>\n'
                '  </thead>\n'
                '  <tbody>\n'
                '    <tr>\n'
                '      <th>0</th>\n'
                '      <td>Equatorial Diameter:</td>\n'
                '      <td>6,792 km</td>\n'
                '    </tr>\n'
                '    <tr>\n'
                '      <th>1</th>\n'
                '      <td>Polar Diameter:</td>\n'
                '      <td>6,752 km</td>\n'
                '    </tr>\n'
                '    <tr>\n'
                '      <th>2</th>\n'
                '      <td>Mass:</td>\n'
                '      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n'
                '    </tr>\n'
                '    <tr>\n'
                '      <t